In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score , cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold


titanic_df = pd.read_csv('/content/drive/MyDrive/ktd231026/m5_ml/train.csv')
titanic_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [30]:
def fillna(titanic_df):
    titanic_df['Age'].fillna(titanic_df['Age'].mean(),inplace=True)
    titanic_df['Cabin'].fillna('N',inplace=True)
    titanic_df['Embarked'].fillna('N',inplace=True)
    titanic_df['Fare'].fillna(0,inplace=True)
    return titanic_df

def drop_features(titanic_df):
    titanic_df.drop(['PassengerId','Name','Ticket'],axis=1,inplace=True)
    return titanic_df

def format_features(titanic_df):
    titanic_df['Cabin'] = titanic_df['Cabin'].str[:1]
    features = ['Cabin','Sex','Embarked']
    for feature in features:
        le = LabelEncoder()
        le = le.fit(titanic_df[feature])
        titanic_df[feature] = le.transform(titanic_df[feature])
    return titanic_df

def transform_features(titanic_df):
    titanic_df = fillna(titanic_df)
    titanic_df = drop_features(titanic_df)
    titanic_df = format_features(titanic_df)
    return titanic_df

In [31]:
titanic_df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [32]:
print(titanic_df.isnull().sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


In [33]:
def fillna(titanic_df):
    titanic_df['Age'].fillna(titanic_df['Age'].mean(), inplace=True)
    titanic_df['Cabin'].fillna('N', inplace=True)
    titanic_df['Embarked'].fillna('N', inplace=True)  # 또는 titanic_df['Embarked'].fillna(titanic_df['Embarked'].mode()[0], inplace=True)
    return titanic_df

def transform_features(titanic_df):
    titanic_df = fillna(titanic_df)
    titanic_df = drop_features(titanic_df)
    titanic_df = format_features(titanic_df)
    return titanic_df

titanic_df = transform_features(titanic_df)

print(titanic_df.isnull().sum())

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Cabin       0
Embarked    0
dtype: int64


In [34]:
# 피처 데이터셋 X, 레이블 데이터셋 y 할당
X = titanic_df.iloc[:, :-1]
y = titanic_df.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=156, stratify=y)

In [35]:
# dt

dt_clf = DecisionTreeClassifier(criterion='gini', max_depth=3, random_state=156)

dt_clf.fit(X_train , y_train)

pred = dt_clf.predict(X_test)
accuracy = accuracy_score(y_test , pred)

print(f'결정 트리 예측 정확도: {accuracy:.4f}','\n')

결정 트리 예측 정확도: 0.7542 



In [36]:
# lr

lr_clf = LogisticRegression()

lr_clf.fit(X_train, y_train)
pred = lr_clf.predict(X_test)
accuracy_lr = accuracy_score(y_test, pred)
print(f'결정 트리 예측 정확도: {accuracy_lr:.4f}','\n')

결정 트리 예측 정확도: 0.7542 



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [37]:
# rf

clf = RandomForestClassifier(n_estimators=100, random_state=42)

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"결정 트리 예측 정확도: {accuracy:.2f}")


결정 트리 예측 정확도: 0.84


In [38]:
# dt_clf , folds=5 적용하여 KFold 교차검증 수행

features = titanic_df.drop('Survived', axis=1)
label = titanic_df['Survived']

dt_clf = DecisionTreeClassifier(random_state=156)

stratified_kfold = StratifiedKFold(n_splits=5)
cv_accuracy = []
print('타이타닉 데이터 세트 크기:', features.shape[0])

n_iter = 0

for train_index, test_index in stratified_kfold.split(features, label):
    X_train, X_test = features.iloc[train_index], features.iloc[test_index]
    y_train, y_test = label.iloc[train_index], label.iloc[test_index]

    dt_clf.fit(X_train, y_train)
    pred = dt_clf.predict(X_test)

    n_iter += 1
    accuracy = np.round(accuracy_score(y_test, pred), 4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]

    print('\n#{0} 교차 검증 정확도 :{1}, 학습 데이터 크기: {2}, 검증 데이터 크기: {3}'.format(n_iter, accuracy, train_size, test_size))
    print('#{0} 검증 세트 인덱스: {1}'.format(n_iter, test_index))

    cv_accuracy.append(accuracy)

print('\n## 평균 검증 정확도:', np.mean(cv_accuracy))

타이타닉 데이터 세트 크기: 891

#1 교차 검증 정확도 :0.7486, 학습 데이터 크기: 712, 검증 데이터 크기: 179
#1 검증 세트 인덱스: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 172 183 184 186 187 190 192 193 194 195 198]

#2 교차 검증 정확도 :0.7697, 학습 데이터 크기: 713, 검증 데이터 크기: 178
#2 검증 세트 인덱스: [168 169 170 171 173 174 175 176 177 178 179 180 181 182 185 188 189 191
 196 197 199 200 201 202 203 204 205 206 207

In [39]:
# dt_clf , cv=5 적용, cross_val_score를 이용하여 교차검증 수행

features = titanic_df.drop('Survived', axis=1)
label = titanic_df['Survived']

dt_clf = DecisionTreeClassifier(random_state=156)

scores = cross_val_score(dt_clf, features, label, scoring='accuracy', cv=5)
print('교차 검증별 정확도:', np.round(scores, 4))
print('평균 검증 정확도:', np.round(np.mean(scores), 4))

교차 검증별 정확도: [0.7486 0.7697 0.7978 0.7809 0.8202]
평균 검증 정확도: 0.7834


GridSearchCV의 최적 하이퍼 파라미터로 학습된 Estimator로 예측 및 평가 수행.

- parameters = {'max_depth':[2,3,5,10], 'min_samples_split':[2,3,5], 'min_samples_leaf':[1,5,8]}
- dt_clf, scoring='accuracy', cv=5 적용

In [40]:
param_grid = {'max_depth':[2,3,5,10], 'min_samples_split':[2,3,5], 'min_samples_leaf':[1,5,8]}

dt_clf = DecisionTreeClassifier(random_state=1)

grid_search = GridSearchCV(dt_clf, param_grid, scoring='accuracy', cv=5)

grid_search.fit(X_train, y_train)

print('GridSearchCV 최적 파라미터:', grid_search.best_params_)
print('GridSearchCV 최고 정확도:', grid_search.best_score_)

GridSearchCV 최적 파라미터: {'max_depth': 10, 'min_samples_leaf': 8, 'min_samples_split': 2}
GridSearchCV 최고 정확도: 0.8078597458879149
